In [4]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Celda 1: Importar librerías
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import os # NUEVO: Importamos 'os' para construir la ruta del archivo de forma segura

# Celda 2: Cargar el clasificador DESDE LA LIBRERÍA de OpenCV
# --- ESTA ES LA PARTE MODIFICADA ---
# 1. Obtenemos la ruta a la carpeta de haarcascades de OpenCV
haarcascade_path = cv2.data.haarcascades

# 2. Unimos esa ruta con el nombre del archivo que necesitamos
cascade_filename = 'haarcascade_frontalface_default.xml'
cascade_path = os.path.join(haarcascade_path, cascade_filename)

# 3. Cargamos el clasificador usando la ruta completa
face_cascade = cv2.CascadeClassifier(cascade_path)

# Comprobamos si el archivo se cargó correctamente
if face_cascade.empty():
    print(f"Error: No se pudo cargar el clasificador Haar desde la ruta: {cascade_path}")
else:
    print("Clasificador de rostros cargado exitosamente desde la librería de OpenCV.")
# --- FIN DE LA PARTE MODIFICADA ---


# Abrir la cámara
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")

# Celda 3: Bucle principal (sin cambios)
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("No se pudo recibir el fotograma. Saliendo...")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            roi_face = frame[y:y+h, x:x+w]
            pixel_level = (20, 20)
            small_face = cv2.resize(roi_face, pixel_level, interpolation=cv2.INTER_LINEAR)
            pixelated_face = cv2.resize(small_face, (w, h), interpolation=cv2.INTER_NEAREST)
            frame[y:y+h, x:x+w] = pixelated_face

        _, jpg_data = cv2.imencode('.jpg', frame)
        image_object = Image(data=jpg_data.tobytes())
        
        display(image_object)
        clear_output(wait=True)

except KeyboardInterrupt:
    print("Detenido por el usuario.")
finally:
    print("Liberando la cámara...")
    cap.release()

Detenido por el usuario.
Liberando la cámara...


In [12]:
# Paso 1: Importar las herramientas que necesitamos
import cv2
import time
import os # Necesario para encontrar el archivo de la librería

# --- INICIO DE LA CONFIGURACIÓN ---

# <<< MANTENEMOS ESTO: Nos encanta el control de FPS que creaste.
TARGET_FPS = 15 # Ajustamos a un valor razonable para esta tarea
tiempo_por_frame = 1 / TARGET_FPS

captura = cv2.VideoCapture(0)

# <<< REEMPLAZAMOS LA LÓGICA DE "FRAME DE REFERENCIA" POR EL "MANUAL DE ROSTROS" ---
# No necesitamos un frame de referencia, necesitamos el clasificador.
# Lo cargamos directamente desde la librería de OpenCV, sin descargar nada.
haarcascade_path = cv2.data.haarcascades
cascade_filename = 'haarcascade_frontalface_default.xml'
cascade_path = os.path.join(haarcascade_path, cascade_filename)
detector_rostros = cv2.CascadeClassifier(cascade_path)

if detector_rostros.empty():
    print("Error: No se pudo cargar el clasificador de rostros.")
# --- FIN DE LA CONFIGURACIÓN ---

while True:
    # <<< MANTENEMOS ESTO: El "chef" (nuestro programa) sigue usando su cronómetro.
    tiempo_inicio_frame = time.time()

    leido, frame_actual = captura.read()
    if not leido:
        break

    # La detección de rostros necesita una imagen en escala de grises para ser eficiente.
    gris = cv2.cvtColor(frame_actual, cv2.COLOR_BGR2GRAY)

    # <<< AQUÍ VIENE EL GRAN CAMBIO: REEMPLAZAMOS EL MOTOR DE DETECCIÓN >>>
    # Ya no comparamos frames ni buscamos contornos.
    # Ahora usamos nuestro "escáner de reconocimiento facial".
    # Le pasamos la imagen en gris y nos devuelve una lista de rostros.
    caras_detectadas = detector_rostros.detectMultiScale(
        image=gris, 
        scaleFactor=1.1, # Qué tanto reducimos la imagen en cada pasada
        minNeighbors=5,  # Cuántos "vecinos" debe tener una detección para ser válida
        minSize=(30, 30) # El tamaño mínimo del rostro a detectar
    )

    # Ahora, en lugar de iterar sobre "contornos", iteramos sobre las "caras detectadas".
    for (x, y, w, h) in caras_detectadas:
        # Dibujamos un rectángulo verde alrededor de cada rostro encontrado.
        cv2.rectangle(img=frame_actual, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
    # <<< FIN DEL CAMBIO EN EL MOTOR DE DETECCIÓN >>>


    # --- INICIO DEL CONTROL DE FPS ---
    # <<< MANTENEMOS ESTO: Esta parte de tu código es excelente y la conservamos intacta.
    tiempo_fin_frame = time.time()
    tiempo_procesamiento = tiempo_fin_frame - tiempo_inicio_frame
    tiempo_espera = tiempo_por_frame - tiempo_procesamiento
    if tiempo_espera > 0:
        time.sleep(tiempo_espera)
        
    fps_reales = 1 / (time.time() - tiempo_inicio_frame)
    cv2.putText(img=frame_actual, 
                text=f"FPS: {int(fps_reales)}", 
                org=(10, 30), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=1, 
                color=(255, 255, 255), 
                thickness=2)
    # --- FIN DEL CONTROL DE FPS ---

    # Mostramos solo la ventana principal. Ya no tenemos una ventana de "Umbral".
    cv2.imshow("Detección de Rostros (con control de FPS)", frame_actual)

    if cv2.waitKey(1) == ord('q'):
        break

# --- LIMPIEZA FINAL ---
captura.release()
cv2.destroyAllWindows()
print("Programa finalizado.")

Programa finalizado.


In [17]:
# Paso 1: Importar las herramientas que necesitamos
import cv2
import time
import os # Necesario para encontrar el archivo de la librería

# --- INICIO DE LA CONFIGURACIÓN ---

# Mantenemos el control de FPS que te gusta.
TARGET_FPS = 20
tiempo_por_frame = 1 / TARGET_FPS

# <<< NUEVO: Un lugar fácil para ajustar la intensidad del pixelado.
# Números más pequeños (ej. 10) = bloques más grandes (más pixelado).
# Números más grandes (ej. 30) = bloques más pequeños (menos pixelado).
INTENSIDAD_PIXELADO = 15

captura = cv2.VideoCapture(0)

# Cargamos el clasificador de rostros desde la librería de OpenCV.
haarcascade_path = cv2.data.haarcascades
cascade_filename = 'haarcascade_frontalface_default.xml'
cascade_path = os.path.join(haarcascade_path, cascade_filename)
detector_rostros = cv2.CascadeClassifier(cascade_path)

if detector_rostros.empty():
    print("Error: No se pudo cargar el clasificador de rostros.")
# --- FIN DE LA CONFIGURACIÓN ---

while True:
    # El cronómetro para controlar los FPS.
    tiempo_inicio_frame = time.time()

    leido, frame_actual = captura.read()
    if not leido:
        break

    # Convertimos a gris para la detección.
    gris = cv2.cvtColor(frame_actual, cv2.COLOR_BGR2GRAY)

    # Detectamos los rostros en la imagen.
    caras_detectadas = detector_rostros.detectMultiScale(
        image=gris, 
        scaleFactor=1.1, 
        minNeighbors=5,  
        minSize=(30, 30)
    )

    # <<< ¡AQUÍ ESTÁ EL CAMBIO CLAVE! >>>
    # Iteramos sobre cada rostro detectado para APLICAR EL PIXELADO.
    for (x, y, w, h) in caras_detectadas:
        # Ya no dibujamos un rectángulo. En su lugar, hacemos el proceso de pixelado:

        # 1. Recortamos el área del rostro de la imagen original a color.
        rostro_recortado = frame_actual[y:y+h, x:x+w]

        # 2. Encogemos el rostro a un tamaño diminuto.
        # Usamos nuestra variable de configuración para la intensidad.
        rostro_pequeno = cv2.resize(
            rostro_recortado, 
            (INTENSIDAD_PIXELADO, INTENSIDAD_PIXELADO), 
            interpolation=cv2.INTER_LINEAR
        )

        # 3. Agrandamos esa imagen pequeña de vuelta a su tamaño original.
        # Usamos INTER_NEAREST para que se vea como bloques y no borroso.
        rostro_pixelado = cv2.resize(
            rostro_pequeno, 
            (w, h), 
            interpolation=cv2.INTER_NEAREST
        )

        # 4. Pegamos el rostro pixelado de vuelta sobre la imagen original.
        frame_actual[y:y+h, x:x+w] = rostro_pixelado
    # <<< FIN DEL CAMBIO >>>


    # --- INICIO DEL CONTROL DE FPS ---
    # Esta parte se mantiene exactamente igual.
    tiempo_fin_frame = time.time()
    tiempo_procesamiento = tiempo_fin_frame - tiempo_inicio_frame
    tiempo_espera = tiempo_por_frame - tiempo_procesamiento
    if tiempo_espera > 0:
        time.sleep(tiempo_espera)
        
    fps_reales = 1 / (time.time() - tiempo_inicio_frame)
    cv2.putText(img=frame_actual, 
                text=f"FPS: {int(fps_reales)}", 
                org=(10, 30), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=1, 
                color=(255, 255, 255), 
                thickness=2)
    # --- FIN DEL CONTROL DE FPS ---

    cv2.imshow("Detección y Pixelado de Rostros", frame_actual)

    if cv2.waitKey(1) == ord('q'):
        break

# --- LIMPIEZA FINAL ---
captura.release()
cv2.destroyAllWindows()
print("Programa finalizado.")

Programa finalizado.
